# 01 – EDA and Baseline Setup

In [1]:
import sys 
from pathlib import Path 

# Point ROOT to /model 
ROOT = Path("..").resolve()

# Add ROOT to python path so `import src... works
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

ROOT

PosixPath('/Volumes/easystore/Projects/sportiq-app/model')

## 1. Imports and Paths

In [2]:
import pandas as pd

# Load paths
from src.paths import (
    RAW_DIR,
    RAW_NBA_SCHEDULE_DIR,
    INTERIM_DIR,
    PROCESSED_DIR,
    MODEL_DIR,
)

RAW_NBA_SCHEDULE_DIR, RAW_NBA_SCHEDULE_DIR.exists(), list(RAW_NBA_SCHEDULE_DIR.iterdir())[:3]

(PosixPath('/Volumes/easystore/Projects/sportiq-app/model/data/raw/NBA_schedule_results'),
 True,
 [PosixPath('/Volumes/easystore/Projects/sportiq-app/model/data/raw/NBA_schedule_results/2016-17_NBA'),
  PosixPath('/Volumes/easystore/Projects/sportiq-app/model/data/raw/NBA_schedule_results/.DS_Store'),
  PosixPath('/Volumes/easystore/Projects/sportiq-app/model/data/raw/NBA_schedule_results/2021-22_NBA')])

## 2. Load Raw Data

In [3]:
all_games = []

# Loop through each season in folder 
for season_dir in sorted(RAW_NBA_SCHEDULE_DIR.glob("*NBA")):
    print("Loading season:", season_dir.name)

    # Loop through each month 
    for month_file in sorted(season_dir.glob("*.xls")):
        print(" -", month_file.name)

        tables = pd.read_html(month_file)
        df = tables[0]

        df["season"] = season_dir.name
        df["source_file"] = month_file.name
        all_games.append(df)

# Combine into one big table 
games = pd.concat(all_games, ignore_index=True)

Loading season: 2015-16_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jun.xls
 - mar.xls
 - may.xls
 - nov.xls
 - oct.xls
Loading season: 2016-17_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jun.xls
 - mar.xls
 - may.xls
 - nov.xls
 - oct.xls
Loading season: 2017-18_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jun.xls
 - mar.xls
 - may.xls
 - nov.xls
 - oct.xls
Loading season: 2018-19_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jun.xls
 - mar.xls
 - may.xls
 - nov.xls
 - oct.xls
Loading season: 2019-20_NBA
 - aug.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jul.xls
 - mar.xls
 - nov.xls
 - oct_2019.xls
 - oct_2020.xls
 - sep.xls
Loading season: 2020-21_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jul.xls
 - jun.xls
 - mar.xls
 - may.xls
Loading season: 2021-22_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jun.xls
 - mar.xls
 - may.xls
 - nov.xls
 - oct.xls
Loading season: 2022-23_NBA
 - apr.xls
 - dec.xls
 - feb.xls
 - jan.xls
 - jun.xls
 - mar.xls
 - may.

## 3. Basic Sanity Checks

In [4]:
games.head()
games.shape[0]
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11525 entries, 0 to 11524
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             11525 non-null  object 
 1   Start (ET)       11525 non-null  object 
 2   Visitor/Neutral  11525 non-null  object 
 3   PTS              11525 non-null  int64  
 4   Home/Neutral     11525 non-null  object 
 5   PTS.1            11525 non-null  int64  
 6   Unnamed: 6       11525 non-null  object 
 7   Unnamed: 7       631 non-null    object 
 8   Attend.          11341 non-null  float64
 9   LOG              11500 non-null  object 
 10  Arena            11525 non-null  object 
 11  Notes            108 non-null    object 
 12  season           11525 non-null  object 
 13  source_file      11525 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.2+ MB


## 4. Data Cleaning 

In [5]:
# Clean column names 

games.columns = (
    games.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w_]+", "", regex=True)
)
games.columns

Index(['date', 'start_et', 'visitorneutral', 'pts', 'homeneutral', 'pts1',
       'unnamed_6', 'unnamed_7', 'attend', 'log', 'arena', 'notes', 'season',
       'source_file'],
      dtype='object')

In [6]:
# Rename important columns 
games = games.rename(columns={
    "visitorneutral": "away_team",
    "pts": "away_pts",
    "homeneutral": "home_team",
    "pts1": "home_pts",
})

# Drop irrelevant columns 
games = games.drop(columns=["unnamed_6", "unnamed_7", "notes", "log"])

# Create a clean date field 
games["date"] = pd.to_datetime(games["date"])
games = games.sort_values("date").reset_index(drop=True)

## 5. Create Target (home_win)

In [7]:
games["home_win"] = (games["home_pts"] > games["away_pts"]).astype(int)

# Verify the target 
games["home_win"].value_counts(normalize=True)

home_win
1    0.569978
0    0.430022
Name: proportion, dtype: float64

## 6. Quick Baseline Model Using Moneyline

### **Step 1:** Baseline Accuracy

In [8]:
baseline_accuracy = games["home_win"].mean()
baseline_accuracy

np.float64(0.5699783080260303)

### **Step 2:** Train/Test Split

In [9]:
from sklearn.model_selection import train_test_split

X = games[["home_pts", "away_pts"]]
y = games["home_win"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

### **Step 3:** Fit Logistic Regression 

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


### **Step 4:** Evaluate the Baseline ML Model

In [11]:
from sklearn.metrics import accuracy_score, roc_auc_score

preds = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, proba)

acc, auc

(1.0, 1.0)

#### **Step 5:** Create the Non-cheating Baseline

In [12]:
games = games.drop(columns=["home_pts", "away_pts"], errors="ignore")
games.head()

,date,start_et,away_team,home_team,attend,arena,season,source_file,home_win
0,2015-10-27,8:00p,Cleveland Cavaliers,Chicago Bulls,21957.0,United Center,2015-16_NBA,oct.xls,1
1,2015-10-27,8:00p,Detroit Pistons,Atlanta Hawks,19187.0,Philips Arena,2015-16_NBA,oct.xls,0
2,2015-10-27,10:30p,New Orleans Pelicans,Golden State Warriors,19596.0,Oracle Arena,2015-16_NBA,oct.xls,1
3,2015-10-28,7:00p,Washington Wizards,Orlando Magic,18846.0,Amway Center,2015-16_NBA,oct.xls,0
4,2015-10-28,7:30p,Philadelphia 76ers,Boston Celtics,18624.0,TD Garden,2015-16_NBA,oct.xls,1


In [14]:
# Build team-centric long table 
home_df = games[['date', 'home_team', 'away_team', 'home_win']].copy()
home_df['team'] = home_df['home_team']
home_df['opponents'] = home_df['away_team']
home_df['is_home'] = 1
home_df['win'] = home_df['home_win']

away_df = games[['date', 'away_team', 'home_team', 'home_win']].copy()
away_df['team'] = away_df['away_team']
away_df['opponent'] = away_df['home_team']
away_df['is_home'] = 0
away_df['win'] = 1 - away_df['home_win']

team_games = pd.concat([home_df, away_df], ignore_index=True)

team_games = team_games.sort_values("date").reset_index(drop=True)
team_games.head()

,date,home_team,away_team,home_win,team,opponents,is_home,win,opponent
0,2015-10-27,Chicago Bulls,Cleveland Cavaliers,1,Chicago Bulls,Cleveland Cavaliers,1,1,NaN
1,2015-10-27,Atlanta Hawks,Detroit Pistons,0,Atlanta Hawks,Detroit Pistons,1,0,NaN
2,2015-10-27,Golden State Warriors,New Orleans Pelicans,1,Golden State Warriors,New Orleans Pelicans,1,1,NaN
3,2015-10-27,Atlanta Hawks,Detroit Pistons,0,Detroit Pistons,NaN,0,1,Atlanta Hawks
4,2015-10-27,Golden State Warriors,New Orleans Pelicans,1,New Orleans Pelicans,NaN,0,0,Golden State Warriors


### **Step 6:** Engineer Rolling Stats

In [15]:
# Compute rolling win percentage 
team_games['win_pct_10'] = (
    team_games.groupby('team')['win']
    .rolling(window=10, min_periods=1, closed='left')
    .mean()
    .reset_index(level=0, drop=True)
)

# Rolling point difference 
team_games['point_diff'] = (
    team_games.apply(
        lambda row: row['win'] - (1-row['win']), axis=1
    )
)

team_games['avg_pd_10'] = (
    team_games.groupby('team')['point_diff']
    .rolling(window=10, min_periods=1, closed='left')
    .mean()
    .reset_index(level=0, drop=True)
)

team_games.head()

,date,home_team,away_team,home_win,team,opponents,is_home,win,opponent,win_pct_10,point_diff,avg_pd_10
0,2015-10-27,Chicago Bulls,Cleveland Cavaliers,1,Chicago Bulls,Cleveland Cavaliers,1,1,NaN,NaN,1,NaN
1,2015-10-27,Atlanta Hawks,Detroit Pistons,0,Atlanta Hawks,Detroit Pistons,1,0,NaN,NaN,-1,NaN
2,2015-10-27,Golden State Warriors,New Orleans Pelicans,1,Golden State Warriors,New Orleans Pelicans,1,1,NaN,NaN,1,NaN
3,2015-10-27,Atlanta Hawks,Detroit Pistons,0,Detroit Pistons,NaN,0,1,Atlanta Hawks,NaN,1,NaN
4,2015-10-27,Golden State Warriors,New Orleans Pelicans,1,New Orleans Pelicans,NaN,0,0,Golden State Warriors,NaN,-1,NaN


### **Step 7:** Join Features Back to Game Rows

In [16]:
# Merge home features 
home_features = (
    team_games[team_games['is_home'] == 1]
    .rename(columns={'win_pct_10': 'home_win_pct_10',
                     'avg_pd_10': 'home_avg_pd_10'})
)
games = games.merge(
    home_features[['date', 'team', 'home_win_pct_10', 'home_avg_pd_10']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)

# Merge away features
away_features = (
    team_games[team_games['is_home'] == 0]
    .rename(columns={'win_pct_10': 'away_win_pct_10',
                     'avg_pd_10': 'away_avg_pd_10'})
)
games = games.merge(
    away_features[['date', 'team', 'away_win_pct_10', 'away_avg_pd_10']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)

# Drop merge helper columns 
games = games.drop(columns=['team_x', 'team_y'], errors='ignore')


### **Step 8:** Rebuild Baseline Model with Real (non-cheating) Features

In [17]:
feature_cols = [
    "home_win_pct_10",
    "away_win_pct_10",
    "home_avg_pd_10",
    "away_avg_pd_10",
]

games_model = games.dropna(subset=feature_cols)

X = games_model[feature_cols]
y = games_model['home_win']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

model = LogisticRegression()
model.fit(X_train, y_train)

preds = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, preds))
print("ROC AUC:", roc_auc_score(y_test, proba))

Accuracy: 0.610338835794961
ROC AUC: 0.6384022949749254


## 7. Save processed table